In [1]:
from gapfinder.feedback import create_roi_vs_contours_images

# get the contents of the poi folder, but only the folder names
poi_folder = f"./output/processed_images"
roi_folder = f"./images/roi_images"
metadata_csv = f"{roi_folder}/roi_metadata.csv"
clear_existing = True

create_roi_vs_contours_images( metadata_csv, roi_folder, poi_folder, clear_existing)

In [2]:
print("hello")

hello
